In [24]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import re

from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.python.framework import dtypes


In [2]:
tf.enable_eager_execution()

In [10]:
N_BALL_SAMPS = 71709
OUTERMOST_SPHERE_SHAPE = [49, 97]

In [129]:
def get_data_queues(train_dir, fake_data=False, shuffle=True, num_epochs=None,
                    num_expected_examples=None):

    print('get_data_queues: num_epochs =', num_epochs, type(num_epochs))

    yamlRE = re.compile(r'.+_.+_[0123456789]+\.yaml')
    recList = []
    if not fake_data:
        for fn in os.listdir(train_dir):
            if yamlRE.match(fn):
                words = fn[:-5].split('_')
                base = words[0]
                idx = int(words[2])
                featureFName = os.path.join(train_dir,
                                            '%s_rotBallSamp_%d.doubles' % (base, idx))
                labelFName = os.path.join(train_dir,
                                          '%s_rotEdgeSamp_%d.doubles' % (base, idx))

                recList.append('%s,%s' % (featureFName, labelFName))

    assert len(recList) == num_expected_examples, ('Found %s examples, expected %d'
                                                   % (len(recList),
                                                      num_expected_examples))

    print('get_data_queues: len(recList) =', len(recList))
    print('get_data_queues: recList[:10] =', recList[:10])


    featureNameT, labelNameT = tf.decode_csv(recList, [[""], [""]],
                                                name='decodeCSV')
    print("feature",featureNameT, "label",labelNameT)
   #namePairQ = tf.train.slice_input_producer([featureNameT, labelNameT],
               #                               shuffle=shuffle,
                       #                       num_epochs=num_epochs)
    feature_label_pair_list = tf.stack([featureNameT, labelNameT], axis=-1)
    print("=== dataset aligned === ",feature_label_pair_list)
    
    
   # dataset = tf.data.Dataset.from_tensor_slices((featureNameT,labelNameT))
    #dataset = feature_label_pair_list
    dataset = tf.data.Dataset.from_tensor_slices(feature_label_pair_list)
    print("=== dataset before shuffling is === ",dataset)
   # print(list(dataset.take(1)))
   # print(list(dataset.take(2)))

    namePairQ = dataset.shuffle(num_expected_examples).repeat(num_epochs)
    print("name pair q ",namePairQ)
    return namePairQ

In [140]:
def read_pair_of_files(namePairQ):
    print('read_pair_of_files: namePairQ[0] =', namePairQ[0])
    print('read_pair_of_files: namePairQ[1] =', namePairQ[1])

    fString = tf.read_file(namePair.numpy()[0], name='featureReadFile')
    fVals = tf.to_float(tf.reshape(tf.decode_raw(fString,
                                     dtypes.float64,
                                     name='featureDecode'),
                                   [N_BALL_SAMPS]),
                        name='featureToFloat')
    lString = tf.read_file(namePair.numpy()[1], name='labelReadFile')
    lVals = tf.to_float(tf.decode_raw(lString, dtypes.float64,
                                      name='labelDecode'),
                        name='labelToFloat')
    nRows, nCols = OUTERMOST_SPHERE_SHAPE
    nOuterSphere = nRows * nCols
    lVals = tf.cond(tf.less(tf.reduce_max(lVals), 1.0e-12),
                    lambda: tf.constant(1.0/float(nOuterSphere),
                                        dtype=dtypes.float32,
                                        shape=[nOuterSphere]),
                    lambda: tf.nn.l2_normalize(lVals, 0))
    lVals = tf.reshape(lVals, OUTERMOST_SPHERE_SHAPE)

    print('read_pair_of_files: fVals, lVals =', fVals, lVals)

    return fVals, lVals

In [141]:
train_dir = 'test'
num_epochs = 1
num_expected_examples = 2
read_threads = 1
namePairQ= get_data_queues(train_dir, shuffle=True, num_epochs=num_epochs,
                               num_expected_examples=num_expected_examples)
flPairList = [read_pair_of_files(namePairQ) for _ in range(read_threads)]

get_data_queues: num_epochs = 1 <class 'int'>
get_data_queues: len(recList) = 2
get_data_queues: recList[:10] = ['test/block_rotBallSamp_4137447.doubles,test/block_rotEdgeSamp_4137447.doubles', 'test/block_rotBallSamp_3913814.doubles,test/block_rotEdgeSamp_3913814.doubles']
feature tf.Tensor(
[b'test/block_rotBallSamp_4137447.doubles'
 b'test/block_rotBallSamp_3913814.doubles'], shape=(2,), dtype=string) label tf.Tensor(
[b'test/block_rotEdgeSamp_4137447.doubles'
 b'test/block_rotEdgeSamp_3913814.doubles'], shape=(2,), dtype=string)
=== dataset aligned ===  tf.Tensor(
[[b'test/block_rotBallSamp_4137447.doubles'
  b'test/block_rotEdgeSamp_4137447.doubles']
 [b'test/block_rotBallSamp_3913814.doubles'
  b'test/block_rotEdgeSamp_3913814.doubles']], shape=(2, 2), dtype=string)
=== dataset before shuffling is ===  <DatasetV1Adapter shapes: (2,), types: tf.string>
name pair q  <DatasetV1Adapter shapes: (2,), types: tf.string>


TypeError: 'DatasetV1Adapter' object does not support indexing

In [139]:
namePairQ
namePairQ
print(list(namePairQ.take(1)))

#for label in label_ds.take(10):
 # print(label_names[label.numpy()])

for namePair in namePairQ.take(10):
    print(namePair.numpy()[1])
    break

[<tf.Tensor: id=830, shape=(2,), dtype=string, numpy=
array([b'test/block_rotBallSamp_3913814.doubles',
       b'test/block_rotEdgeSamp_3913814.doubles'], dtype=object)>]
b'test/block_rotEdgeSamp_3913814.doubles'


In [149]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)



AttributeError: 'list' object has no attribute 'numpy'